# Assignment 5 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [1]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.feature_selection import SelectKBest, chi2 ,mutual_info_classif

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [2]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [3]:
# word net installation:

# unmark if you want to use and need to install
# !pip install wn
# !python -m wn download omw-he:1.4

In [4]:
# word net import:

# unmark if you want to use:
# import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [5]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
# !pip install hebrew_tokenizer

In [6]:
# Hebrew tokenizer import:

# unmark if you want to use:
# import hebrew_tokenizer as ht

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [3]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [4]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [5]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

In [42]:
# Method that Removes chars that removes numbers, punctuation marks, multi blank spaces & trail spaces
# -- Used with .apply() method on df_train
# -- Returns: str (string)

def Remove_NonChars(str):
    str = re.sub(r'\d+', '', str) # Removes Numbers
    str = re.sub(r'[^\w\s]', '', str) # Removes punctuation marks
    str = re.sub(r'\s+', ' ', str) # Removes multi blank spaces
    str = str.strip() # Remove trail spaces
    return str

In [48]:
# Method that prepares training data
# -- Returns: X_train, X_test, y_train, y_test

def Prepare_Train_Data(df: pd.DataFrame):
    
    # Remove non-character elements from the 'story' column
    df['story'] = df['story'].apply(Remove_NonChars)

    # Vectorize the text data using TfidfVectorizer
    vec = TfidfVectorizer(ngram_range = (1,1), min_df = 5)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(df['story'], df['gender'], test_size=0.2, random_state=42)

    # Vectorize the training and testing sets
    X_train_vec = vec.fit_transform(X_train)
    X_test_vec = vec.transform(X_test)

    # Perform feature selection using SelectKBest and mutual information scoring
    selector = SelectKBest(mutual_info_classif, k=1000) # chi2 # mutual_info_classif
    selector.fit(X_train_vec, y_train)

    # Apply feature selection to the vectorized data
    X_train_selected = selector.transform(X_train_vec)
    X_test_selected = selector.transform(X_test_vec)

    # Scale the selected features using MinMaxScaler
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train_selected.toarray())
    X_test = scaler.transform(X_test_selected.toarray())
    
    return X_train, X_test, y_train, y_test

In [51]:
# Method that prepares testing data
# -- Returns: df['story'], X_test 

def Prepare_Test_Data(df: pd.DataFrame, y_train: np.array):

     # Remove non-character elements from the 'story' column
     df['story'] = df['story'].apply(Remove_NonChars)

     # Vectorize the text data using TfidfVectorizer
     vec = TfidfVectorizer(ngram_range = (1,1), min_df = 5)

     # Get the test data
     X_test = df['story']

     # Vectorize the test data
     X_test_vec = vec.fit_transform(X_test)

     # Perform feature selection using SelectKBest and mutual information scoring
     selector = SelectKBest(mutual_info_classif, k=1000) 

     # Apply feature selection to the vectorized data
     X_test_selected = selector.fit_transform(X_test_vec, np.zeros(X_test_vec.shape[0]))
     
     # Scale the selected features using MinMaxScaler
     scaler = MinMaxScaler()
     X_test = scaler.fit_transform(X_test_selected.toarray())

     return df['story'], X_test

In [44]:
# Method that gets X_train and y_train as inputs and return best evaluated Logistic Regression Model
# -- Returns: best_model (LogisticRegression)

def Best_LogisticRegression_Model(X_train: np.array, y_train: np.array):

    # Create LogisticRegression Model  & Create train new model based on X_train_scaled and y_train
    LR = linear_model.LogisticRegression()
    LR_params = {'C': [0.1, 1, 10], 'penalty': ['l1', 'l2']}

    # Find best params using GridSearchSV
    grid_search = GridSearchCV(LR, LR_params, cv=5, n_jobs = -1)
    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_

    # Create New Model based on best params and train it with our data
    best_model = linear_model.LogisticRegression(**best_params)
    best_model.fit(X_train, y_train)  # Fit the best model to the training data

    return best_model


In [53]:
###########################################################
###############------[ Main Program ]----##################
###########################################################

#1. Prepare train Data from df_train
X_train, X_test, y_train, y_test = Prepare_Train_Data(df_train)

#2. Train Model based on Train Data
LR_Model = Best_LogisticRegression_Model(X_train, y_train)

#3. Using RepeatedKFold for evaluating the preformance of our model by performing cross-validation on different random splits of data 
cv = model_selection.RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)
score = cross_val_score(LR_Model, X_test, y_test, scoring='accuracy', cv=cv, n_jobs=-1)

#4. Get Model Predictions for X_test
y_pred = LR_Model.predict(X_test)

#5. f1_scores for Male, Female and Average
f1_male = f1_score(y_test, y_pred, pos_label='m')
f1_female = f1_score(y_test, y_pred, pos_label='f')
f1_average =(f1_male + f1_female)/2

#[--- Print Scores ----]
print('Accuracy Score: %.4f | std: %.4f' % (np.mean(score),np.std(score)))
print('F1_Male Score: %.4f' % f1_male)
print('F1_Female Score: %.4f' % f1_female)
print('F1_Average Score: %.4f \n' % f1_average)

#6.[Final]: Testing the Model on df_test
test_Data, test_X_test = Prepare_Test_Data(df_test, y_train)
test_y_pred = LR_Model.predict(test_X_test)
df_predicted = pd.DataFrame({'story': test_Data, 'predicted gender': test_y_pred})

print(df_predicted['predicted gender'].value_counts())

Accuracy Score: 0.7593 | std: 0.0945
F1_Male Score: 0.8790
F1_Female Score: 0.4444
F1_Average Score: 0.6617 

m    261
f     62
Name: predicted gender, dtype: int64


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [34]:
df_predicted.to_csv('classification_results.csv',index=False)